In [ ]:
import sqlite3
from pathlib import Path
from tqdm.auto import tqdm
import re

cache_folder = "../cache/trials"
master_file = Path("../results.db")

In [ ]:
def safe_alias(path: Path) -> str:
    alias = re.sub(r"\W+", "_", path.stem)
    if alias[0].isdigit():
        alias = "a_" + alias
    return alias

def merge_sqlite_dbs(master_path: str, others: list[str]):
    master = sqlite3.connect(master_path, uri=True)
    cur = master.cursor()

    for db in tqdm(others):
        # TODO Check if search id already finished else skip for now

        p = Path(db).resolve()
        alias = safe_alias(p)

        # Proper URI: absolute path + uri=True on main connection
        uri = f"file:{p}?mode=ro"

        cur.execute(f'ATTACH DATABASE ? AS "{alias}"', (uri,))

        cur.execute("BEGIN")

        tables = cur.execute(
            f'SELECT name FROM "{alias}".sqlite_master WHERE type="table"'
        ).fetchall()

        for (table,) in tables:
            cur.execute(
                f'INSERT OR IGNORE INTO "{table}" '
                f'SELECT * FROM "{alias}"."{table}"'
            )

        cur.execute("COMMIT")
        cur.execute(f'DETACH DATABASE "{alias}"')

        # TODO Commit + Delete Cache File

    master.commit()
    master.close()

In [3]:
import pandas as pd
master = sqlite3.connect(master_file)
df = pd.read_sql("Select * FROM trials", master)
master.close()
df

,hash_key,model,search_id,search_space_hash,data_config_hash,data_id,fold,fidelity,config,train_duration,...,cpu_count_available,logical_cpu_count,ram_gb,commit_hash,uncommited_changes,mean_absolute_error,mean_squared_error,r2_score,mean_absolute_percentage_error,root_mean_squared_error
0,4be3b0b12be320d5c5c9ebed4d591bbf0ed62a9247d097...,AdaBoost,5ffe31d1-15dd-4bb6-a51e-c59685743d1d,93523cd6cededff7354d19f4e6683eddb031d99b6b9105...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,2,0,1.0,"{""learning_rate"": 1.133898138999939, ""prepro_m...",0.040970,...,104,208,503,700d8cc225a7d96ad955f41b9785854da6392c8f,0,8269.245265,1.326157e+08,-0.555189,7.603502e+00,11515.888236
1,df29a071bd7a842c303e537a43dbfcf3ea63c6663ff6cf...,AdaBoost,a9051cbc-d932-4d8e-b251-b02a1a619be5,93523cd6cededff7354d19f4e6683eddb031d99b6b9105...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,9,0,1.0,"{""learning_rate"": 1.9690442085266113, ""prepro_...",0.413681,...,104,208,503,700d8cc225a7d96ad955f41b9785854da6392c8f,0,0.608399,6.248655e-01,0.206208,1.075201e-01,0.790484
2,d592b70300b62342ca1b10f0cae8b8c117149a7c4ccd4a...,AdaBoost,e1df87bf-62fa-4c1d-8de7-0ccf0f4afbc8,93523cd6cededff7354d19f4e6683eddb031d99b6b9105...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,20,0,1.0,"{""learning_rate"": 1.1754683256149292, ""prepro_...",0.445963,...,104,208,503,700d8cc225a7d96ad955f41b9785854da6392c8f,0,180911.974416,5.993239e+10,0.424448,6.044169e-01,244810.925099
3,06fcffe39ec57ef827ad8c73227c2cc16efa82ef7785ea...,AdaBoost,2979190c-45e4-4936-870c-c9e2d618e026,93523cd6cededff7354d19f4e6683eddb031d99b6b9105...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,8,0,1.0,"{""learning_rate"": 0.28005942702293396, ""prepro...",0.301506,...,104,208,503,700d8cc225a7d96ad955f41b9785854da6392c8f,0,0.005509,4.269566e-05,0.801058,5.643576e-03,0.006534
4,b278c1b9f1f3ef26390a389b59173859ef2e7e98bb998e...,AdaBoost,6cd82c29-c3c6-46d2-8399-c5f37c36c03f,93523cd6cededff7354d19f4e6683eddb031d99b6b9105...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,3,0,1.0,"{""learning_rate"": 1.7423291206359863, ""prepro_...",0.421640,...,104,208,503,700d8cc225a7d96ad955f41b9785854da6392c8f,0,9.271899,1.345154e+02,0.526834,3.383575e-01,11.598078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394464,78f7c25b577dc9156c7ccf3777b7533fd11773ce511252...,Tweedie,33a27eec-d055-4794-99ef-2a7e0c6e7713,5e1c657ba94c15a54edab34e05b4466fdbdad28e91c76b...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,31,7,1.0,"{""alpha"": 2.967542223475816, ""prepro_max_featu...",2.047605,...,104,208,503,5d4fb8e2a194ed2d395571336c750c6f43ebfb3b,1,1.862118,8.721001e+00,0.219686,1.795965e+15,2.953134
4394465,38cc7b3b4a8ddcd0cc0b95aed1479c57c951a9aa01b957...,Tweedie,33a27eec-d055-4794-99ef-2a7e0c6e7713,5e1c657ba94c15a54edab34e05b4466fdbdad28e91c76b...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,31,7,1.0,"{""alpha"": 7.9755754470825195, ""prepro_max_feat...",0.550174,...,104,208,503,5d4fb8e2a194ed2d395571336c750c6f43ebfb3b,1,2.921467,1.578185e+01,-0.412084,2.264533e+15,3.972637
4394466,2158ab8f51b4c8054669c7440bd05ebc230fd958abf446...,Tweedie,33a27eec-d055-4794-99ef-2a7e0c6e7713,5e1c657ba94c15a54edab34e05b4466fdbdad28e91c76b...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,31,7,1.0,"{""alpha"": 0.17187724922161685, ""prepro_max_fea...",1.914273,...,104,208,503,5d4fb8e2a194ed2d395571336c750c6f43ebfb3b,1,1.917991,8.360544e+00,0.251938,1.729615e+15,2.891461
4394467,178ea031ecf113ffa4ebfac5671b20492d80e3d629fdad...,Tweedie,33a27eec-d055-4794-99ef-2a7e0c6e7713,5e1c657ba94c15a54edab34e05b4466fdbdad28e91c76b...,2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,31,7,1.0,"{""alpha"": 0.2369241634816871, ""prepro_max_feat...",1.826177,...,104,208,503,5d4fb8e2a194ed2d395571336c750c6f43ebfb3b,1,1.900599,8.382831e+00,0.249944,1.739073e+15,2.895312


In [4]:
cache_files = list(Path(cache_folder).glob("*.db"))
# usage
merge_sqlite_dbs(master_file, cache_files)

  0%|          | 0/6452 [00:00<?, ?it/s]

In [5]:
cache_files = list(Path(cache_folder).glob("*.db"))
for file in cache_files:
    file.unlink()